In [1]:
!pip3 install requests beautifulsoup4 pandas scikit-learn lxml --quiet


In [2]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import string


with open('new_mayo_clinic_diseases.csv', 'w', newline='', encoding='utf-8') as f:
    csv_writer = writer(f)
    csv_writer.writerow(['Letter', 'Disease Name','URL'])


    for letter in string.ascii_uppercase:
        while True:
            url = f"https://www.mayoclinic.org/diseases-conditions/index?letter={letter}"
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            disease_links = soup.find_all('a', class_='cmp-anchor--plain cmp-button cmp-button__link cmp-result-name__link')

            if not disease_links:
                break

            for link in disease_links:
                url1 = link.get('href')
                disease_name = link.text.strip()
                csv_writer.writerow([letter, disease_name,url1])
                print(f"Letter {letter} - Added: {disease_name} and URL")

            next_page = soup.find('a', {'aria-label': 'Next page'})
            if not next_page:
                break
print("Scraping completed successfully!")

Letter A - Added: Abdominal aortic aneurysm and URL
Letter A - Added: Absence seizure and URL
Letter A - Added: Acanthosis nigricans and URL
Letter A - Added: Achalasia and URL
Letter A - Added: Achilles tendinitis and URL
Letter A - Added: Achilles tendon rupture and URL
Letter A - Added: ACL injury and URL
Letter A - Added: Acne and URL
Letter A - Added: Acoustic neuroma and URL
Letter A - Added: Acromegaly and URL
Letter A - Added: Actinic keratosis and URL
Letter A - Added: Acute coronary syndrome and URL
Letter A - Added: Acute flaccid myelitis (AFM) and URL
Letter A - Added: Acute kidney injury and URL
Letter A - Added: Acute liver failure and URL
Letter A - Added: Acute lymphocytic leukemia and URL
Letter A - Added: Acute myelogenous leukemia and URL
Letter A - Added: Acute sinusitis and URL
Letter A - Added: Addison's disease and URL
Letter A - Added: Adenomyosis and URL
Letter A - Added: Adjustment disorders and URL
Letter A - Added: Adnexal tumors and masses and URL
Letter A 

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

df = pd.read_csv('new_mayo_clinic_diseases.csv')

def scrape_symptoms(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # find any h2 that contains 'Symptom'
        symptoms_header = soup.find('h2', string=lambda text: text and 'Symptom' in text)
        if symptoms_header:
            ul = symptoms_header.find_next('ul')
            if ul:
                return ul.get_text(separator=' ', strip=True)
        return ""
    except Exception as e:
        return ""

df['Symptoms'] = df['URL'].apply(scrape_symptoms)
df.to_csv('diseases_with_symptoms.csv', index=False)


In [4]:
df.shape


(1167, 4)

In [5]:
df

,Letter,Disease Name,URL,Symptoms
0,A,Abdominal aortic aneurysm,https://www.mayoclinic.org/diseases-conditions...,"Deep, constant pain in the belly area or side ..."
1,A,Absence seizure,https://www.mayoclinic.org/diseases-conditions...,A sudden stop in activity without falling. Lip...
2,A,Acanthosis nigricans,https://www.mayoclinic.org/diseases-conditions...,Insulin resistance. Most people who have acant...
3,A,Achalasia,https://www.mayoclinic.org/diseases-conditions...,"Difficulty swallowing, called dysphagia, which..."
4,A,Achilles tendinitis,https://www.mayoclinic.org/diseases-conditions...,Age. Achilles tendinitis is more common with a...
...,...,...,...,...
1162,X,X-linked agammaglobulinemia,https://www.mayoclinic.org/diseases-conditions...,
1163,Y,Yeast infection (vaginal),https://www.mayoclinic.org/diseases-conditions...,Itching and irritation in the vagina and tissu...
1164,Y,Yips,https://www.mayoclinic.org/diseases-conditions...,Older age. More experience playing golf. Tourn...
1165,Z,Zenker's diverticulum,https://www.mayoclinic.org/diseases-conditions...,"Trouble swallowing, called dysphagia. Burping...."


In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

df = pd.read_csv('new_mayo_clinic_diseases.csv')

def scrape_symptoms_and_rarity(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        symptoms_header = soup.find('h2', string='Symptoms')
        overview_header = soup.find('h2', string='Overview')

        symptoms = []
        if symptoms_header:
            for li in symptoms_header.find_next('ul').find_all('li'):
                symptoms.append(li.get_text(strip=True))

        if overview_header and symptoms_header:
            content = ''
            current = overview_header.find_next()
            while current and current != symptoms_header:
                if current.name == 'p':
                    content += current.get_text() + ' '
                current = current.find_next()
            if re.search(r'\b(rare|uncommon|rarely)\b', content.lower()):
                is_rare = 1

        return symptoms, is_rare
    except:
        return [], 0

df[['Symptoms', 'IsRare']] = df['URL'].apply(
    lambda x: pd.Series(scrape_symptoms_and_rarity(x))
)
df.to_csv('prefinal_diseases_with_symptoms_enhanced.csv', index=False)
     

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

df = pd.read_csv('new_mayo_clinic_diseases.csv')

def scrape_symptoms_and_rarity(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        symptoms_header = soup.find('h2', string='Symptoms')
        overview_header = soup.find('h2', string='Overview')

        symptoms = []
        is_rare = 0  # default

        # --- Extract Symptoms (independent) ---
        if symptoms_header:
            current = symptoms_header.find_next()
            while current and current.name != "h2":  # stop at next section
                if current.name == "ul":
                    for li in current.find_all("li"):
                        symptoms.append(li.get_text(strip=True))
                elif current.name == "p":
                    text = current.get_text(strip=True)
                    if text:
                        symptoms.append(text)
                current = current.find_next()

        # --- Check for rarity keywords (independent) ---
        if overview_header:
            content = ''
            current = overview_header.find_next()
            while current and current.name != "h2":  # stop at next section
                if current.name == 'p':
                    content += current.get_text() + ' '
                current = current.find_next()
            if re.search(r'\b(rare|uncommon|rarely)\b', content.lower()):
                is_rare = 1

        return symptoms, is_rare

    except Exception as e:
        return [], 0

# Apply function
df[['Symptoms', 'IsRare']] = df['URL'].apply(
    lambda x: pd.Series(scrape_symptoms_and_rarity(x))
)

df.to_csv('final_diseases_with_symptoms_enhanced.csv', index=False)
print("✅ Done. Now all diseases will have Symptoms, and rarity is flagged separately.")


✅ Done. Now all diseases will have Symptoms, and rarity is flagged separately.


In [ ]:
import pandas as pd
import ast
import itertools

# Load your current CSV
df = pd.read_csv("final_diseases_with_symptoms_enhanced.csv")

# Convert the string representation of list back into actual Python list
df["Symptoms"] = df["Symptoms"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Create a new dataframe for transformed rows
rows = []

for _, row in df.iterrows():
    symptoms = row["Symptoms"]
    disease = row["Disease Name"]
    
    # Generate all non-empty combinations of symptoms
    for r in range(1, len(symptoms) + 1):
        for combo in itertools.combinations(symptoms, r):
            rows.append([list(combo), disease])

# Convert back to DataFrame
df_transformed = pd.DataFrame(rows, columns=["Symptoms", "Disease Name"])

# Save the transformed CSV
df_transformed.to_csv("DiseaseAndSymptoms_Transformed.csv", index=False)

